In [1]:
%pip install sentence-transformers
%pip install tf-keras
%pip install tensorflow-datasets


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sentence_transformers import SentenceTransformer
import tensorflow_datasets as tfds
import tensorflow as tf

/Users/nithesh-21075/Work/kaggle/Gen-AI-Intensive-Course/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_NAME = "all-MiniLM-L6-v2"
TASK_TYPE = "RETRIEVAL_DOCUMENT"
TITLE = "SentenceTransformers"
TEXT = "Embed Text"

In [4]:
embedding_vx = SentenceTransformer(MODEL_NAME)

In [5]:
def LLM_embed(text):
    def embed_text(text):
        text_str = text.numpy().decode('utf-8')
        return embedding_vx.encode(text_str)

    output = tf.py_function(func=embed_text, inp=[text], Tout=tf.float32)
    output.set_shape((384,))
    return output

In [ ]:
train_data = tfds.load("imdb_reviews", split="train", as_supervised=True)

import numpy as np

print("Extracting text from dataset...")
train_texts = []
train_labels = []
for text, label in tfds.as_numpy(train_data):
    train_texts.append(text.decode('utf-8'))
    train_labels.append(label)

train_labels = np.array(train_labels)

print("Encoding texts (this may take a few minutes)...")
train_embeddings = embedding_vx.encode(train_texts, batch_size=128, show_progress_bar=True)

print("Creating optimized dataset...")
optimized_dataset = tf.data.Dataset.from_tensor_slices((train_embeddings, train_labels))
optimized_dataset = optimized_dataset.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(384,)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(optimized_dataset, epochs=10)

Extracting text from dataset...
